In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.stats import binned_statistic
from scipy.stats import binned_statistic_2d
from scipy.optimize import curve_fit

In [ ]:
cols = ["# y_exit", "z_exit", "x_start", "y_start", "z_start", "px_start", "py_start", "pz_start"]
#df = pd.concat([pd.read_csv(f"data/simulation/job_{i}.txt", sep="\t", header=0, usecols=cols) for i in range(24)], ignore_index=True)
df = pd.read_csv("data/simulation/job_23.txt", sep="\t", header=0, usecols=cols)
job = 23
for i in range(49):
    job+=24
    df = pd.concat([df,pd.read_csv(f"data/simulation/job_{job}.txt", sep="\t", header=0, usecols=cols)],ignore_index=True)
df
#df.to_hdf("data/simdata.h5",key="simulation")

In [ ]:
df = pd.read_hdf("data/simdata.h5")
point=np.arange(1,25)
fiber=np.arange(1,51)

print(point,fiber)
df

In [ ]:
# a) calculate the angle theta, using the components of momentum

px = df["px_start"].to_numpy()
py = df["py_start"].to_numpy()
pz = df["pz_start"].to_numpy()

x = df["x_start"].to_numpy()
y = df["y_start"].to_numpy()
z = df["z_start"].to_numpy()

p = px**2+py**2+pz**2
theta=np.rad2deg(np.arccos(px))

plt.hist(theta)
plt.yscale("log")

In [ ]:
# b) function definition for calculating distance of skew lines

def r_min(x0,p0): # c0 + c*y, x0 + p0*x
    # centerline of x-axis
    c = np.array([1,0,0])
    
    # calculate normal vector between photon line and x-axis
    n = np.cross(p0,c)
    
    # scalar product of starting point and normal vector
    E = np.dot(n,-x0)
    
    # Normalizing to magnitude of normal vector
    NF = E/np.linalg.norm(n)
    
    # Absolute value gives minimum distance
    r_min = np.abs(NF)
    
    return r_min

# More rigorous code from StackExchange as saity check, includes special cases
def closestDistanceBetweenLines(a0,a1,b0=np.array([0,0,0]),b1=np.array([2400,0,0]),clampAll=False,clampA0=False,clampA1=False,clampB0=False,clampB1=False):

    ''' Given two lines defined by numpy.array pairs (a0,a1,b0,b1)
        Return the closest points on each segment and their distance
    '''

    # If clampAll=True, set all clamps to True
    if clampAll:
        clampA0=True
        clampA1=True
        clampB0=True
        clampB1=True


    # Calculate denomitator
    A = a1 - a0
    B = b1 - b0
    magA = np.linalg.norm(A)
    magB = np.linalg.norm(B)
    
    _A = A / magA
    _B = B / magB
    
    cross = np.cross(_A, _B);
    denom = np.linalg.norm(cross)**2
    
    
    # If lines are parallel (denom=0) test if lines overlap.
    # If they don't overlap then there is a closest point solution.
    # If they do overlap, there are infinite closest positions, but there is a closest distance
    if not denom:
        d0 = np.dot(_A,(b0-a0))
        
        # Overlap only possible with clamping
        if clampA0 or clampA1 or clampB0 or clampB1:
            d1 = np.dot(_A,(b1-a0))
            
            # Is segment B before A?
            if d0 <= 0 >= d1:
                if clampA0 and clampB1:
                    if np.absolute(d0) < np.absolute(d1):
                        return a0,b0,np.linalg.norm(a0-b0)
                    return a0,b1,np.linalg.norm(a0-b1)
                
                
            # Is segment B after A?
            elif d0 >= magA <= d1:
                if clampA1 and clampB0:
                    if np.absolute(d0) < np.absolute(d1):
                        return a1,b0,np.linalg.norm(a1-b0)
                    return a1,b1,np.linalg.norm(a1-b1)
                
                
        # Segments overlap, return distance between parallel segments
        return None,None,np.linalg.norm(((d0*_A)+a0)-b0)
        
    
    
    # Lines criss-cross: Calculate the projected closest points
    t = (b0 - a0);
    detA = np.linalg.det([t, _B, cross])
    detB = np.linalg.det([t, _A, cross])

    t0 = detA/denom;
    t1 = detB/denom;

    pA = a0 + (_A * t0) # Projected closest point on segment A
    pB = b0 + (_B * t1) # Projected closest point on segment B


    # Clamp projections
    if clampA0 or clampA1 or clampB0 or clampB1:
        if clampA0 and t0 < 0:
            pA = a0
        elif clampA1 and t0 > magA:
            pA = a1
        
        if clampB0 and t1 < 0:
            pB = b0
        elif clampB1 and t1 > magB:
            pB = b1
            
        # Clamp projection A
        if (clampA0 and t0 < 0) or (clampA1 and t0 > magA):
            dot = np.dot(_B,(pA-b0))
            if clampB0 and dot < 0:
                dot = 0
            elif clampB1 and dot > magB:
                dot = magB
            pB = b0 + (_B * dot)
    
        # Clamp projection B
        if (clampB0 and t1 < 0) or (clampB1 and t1 > magB):
            dot = np.dot(_A,(pB-a0))
            if clampA0 and dot < 0:
                dot = 0
            elif clampA1 and dot > magA:
                dot = magA
            pA = a0 + (_A * dot)

    
    return pA,pB,np.linalg.norm(pA-pB)

In [ ]:
p0 = np.stack((px,py,pz), axis=-1)
x0 = np.stack((x,y,z), axis=-1)

rmin = np.empty(len(p0))
rmin2 = np.empty(len(p0))

for i in range(len(p0)):
    # both functions are incredibly slow for large datasets, this might run for a while
    _,_,rmin2[i] = closestDistanceBetweenLines(x0[i],(x0[i]+p0[i]))
    rmin[i] = r_min(x0[i],p0[i])

print(rmin)
print(np.any(np.isclose(rmin,rmin2))==False)

In [ ]:
# Sanity check if skew line distance is calculated correctly
print(rmin)
print(rmin2)
np.any(np.isclose(rmin,rmin2)==False)

In [ ]:
bins_theta = np.linspace(0,180,21)
bins_rmin = np.linspace(0,1.1,21)
A, bt, br = np.histogram2d(theta, rmin, bins=[bins_theta, bins_rmin])
plt.hist2d(theta,rmin,bins=[bt,br],cmap="viridis",norm=LogNorm())
plt.colorbar()

In [ ]:
# c) determine mean r_min for each bin in theta
stat,edge,no  = binned_statistic(theta,rmin,bins=bins_theta)
mid = edge[:-1] + np.diff(edge)/2

In [ ]:
# c) Gaussian distribution fit function for mean r_min
gauss = lambda x,a,mu,sigma,c: a/np.sqrt(2*np.pi*sigma**2) * np.exp(-(x-mu)**2/sigma**2) + c

gauss_par, gauss_cov = curve_fit(gauss,mid,stat,p0=[1,100,50,0.06],sigma=None,absolute_sigma=False)

In [ ]:
linplt = np.linspace(0,180,200)
plt.stairs(stat, edge, color="b")
plt.plot(linplt,gauss(linplt,*gauss_par))

In [ ]:
# d) calculate r_theta
r_core = 0.25/2
theta_refl = np.rad2deg(np.arcsin(np.sqrt(1- (rmin[rmin<=r_core]**2/ r_core**2))*np.sin(np.deg2rad(theta[rmin<=r_core]))))#gauss(theta,*gauss_par)**2
bins_theta_refl = np.linspace(0,180,21)
plt.hist(theta_refl,bins=bins_theta_refl)
plt.yscale("log")

In [ ]:
# d) determine mean theta_refl for each bin in theta
stat_refl, edge_refl, no_refl = binned_statistic(theta[rmin<=r_core],theta_refl)
plt.stairs(stat_refl, edge_refl, color="b")

In [ ]:
# e) calculate distance to fibre centerline
y_ex = df["# y_exit"].to_numpy()
z_ex = df["z_exit"].to_numpy()

r_ex = np.sqrt(y_ex**2 + z_ex**2)
#plt.hist(r_ex, bins=20)

# e) create mask for determining if photon exited from core or cladding (thrown out if neither)
mask_clad = np.logical_and(r_ex<=0.25/2,r_ex>0.22/2)
mask_core = r_ex<=0.22/2

# e) get arrays of exit radii and theta for core/cladding photons
r_ex_cladding = r_ex[mask_clad]
theta_cladding = theta[mask_clad]
r_ex_core = r_ex[mask_core]
theta_core = theta[mask_core]

In [ ]:
# e) show number of core/cladding photons binned in theta
stat_core, edge_core, no_core = binned_statistic(theta_core,r_ex_core,bins=bins_theta,statistic="count")
stat_clad, edge_clad, no_clad = binned_statistic(theta_cladding,r_ex_cladding,bins=bins_theta,statistic="count")

plt.stairs(stat_core, edge_core, color="b")
plt.stairs(stat_clad, edge_clad, color="r")
plt.yscale("log")
#plt.ylim((0,100))

In [ ]:
mask_inside = r_ex<=0.25/2
mask_rmin1 = np.logical_and(mask_inside,rmin<0.22/2)
stat_photon, edge_photon, no_photon = binned_statistic(theta[mask_rmin1], r_ex[mask_rmin1], bins=bins_theta, statistic="count")
plt.stairs(stat_photon, edge_photon, color="b")
plt.yscale("log")
